In [1]:
import numpy as np
import random
import xlrd
import math
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, roc_curve, silhouette_score
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [2]:
#Save or read variables
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
  
def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

In [3]:
data_path='./data/'
DSS1 = np.loadtxt(data_path + 'DSS1.txt')
DSS2 = np.loadtxt(data_path + 'DSS2.txt')
#Integrated semantic similarity for diseases
DSS = (DSS1 + DSS2) / 2
#Gaussian interaction profile kernel similarity for diseases
DGS = np.loadtxt(data_path + 'DGS.txt')
#Integrated similarity for diseases
IDS = np.zeros(shape = (DSS.shape[0], DSS.shape[1]))
for i in range(DSS.shape[0]):
    for j in range(DSS.shape[1]):
        if DSS[i][j] == 0:
            IDS[i][j] = DGS[i][j]
        else:       
            IDS[i][j] = DSS[i][j]
#Functional similarity for miRNAs          
MFS = np.loadtxt(data_path + 'MFS.txt')
#Gaussian interaction profile kernel similarity for miRNAs
MGS = np.loadtxt(data_path + 'MGS.txt')
#Integrated similarity for miRNAs
IMS = np.zeros(shape = (MFS.shape[0], MFS.shape[1]))
for i in range(MFS.shape[0]):
    for j in range(MFS.shape[1]):
        if MFS[i][j] == 0:
            IMS[i][j] = MGS[i][j]
        else:
            IMS[i][j] = MFS[i][j]
#miRNA-disease associations matrix
MD = np.zeros(shape = (DSS.shape[0], MFS.shape[0]))
asso_file =  xlrd.open_workbook(data_path + 'Human miRNA-disease associations.xlsx')
asso_pairs = asso_file.sheets()[0]
for i in range(asso_pairs.nrows):
    asso = asso_pairs.row_values(i)
    m = int(asso[0])
    n = int(asso[1])
    MD[n-1,m-1]=1
#Verified miRNA disease pair
known=[]
#Unverified miRNA disease pair
unknown=[]
for x in range(MD.shape[0]):
    for y in range(MD.shape[1]):
        if MD[x,y]==0:
            unknown.append((x,y))
        else:
            known.append((x,y))

In [14]:
#position sample set
posi_list = []
#unlabeled sample set
unlabelled_list = []
#total sample set
all_list = []

for i in range(len(known)):
    posi=IDS[known[i][0],:].tolist() + IMS[known[i][1],:].tolist()
    posi_list.append(posi)
    all_list.append(posi)

for i in range(len(unknown)):
    unlabelled=IDS[unknown[i][0],:].tolist() + IMS[unknown[i][1],:].tolist()
    unlabelled_list.append(unlabelled)
    all_list.append(unlabelled)

#The total sample set is disordered to avoid the influence of order on the clustering results
random.shuffle(all_list)

In [15]:
#Implement MiniBatchKMeans clustering algorithm ten times for total sample set

#Record the number of times each sample is divided into a certain cluster
sum = np.zeros(len(all_list),dtype=int)
#Record the final cluster of each sample
final = np.zeros(len(all_list),dtype=int)
for i in range(10):
    #According to our experiment, we set the number of clusters to 2
    cls = MiniBatchKMeans(n_clusters=2,batch_size=3072).fit(all_list)
    yhat = cls.predict(all_list)
    #According to the multiple clustering results of the total sample set, the total sample set is divided into subsets A and B
    #Among the two subsets A and B, subset A (B) is always greater than the size of subset A (B)
    #In order to ensure that subsets a (b) can obtain the same label after each clustering: 
    #suppose that the label of small subset is 1 and the label of large subset is 0
    #After clustering, when the number of samples with label 1 is greater than the number of samples with label 0, 
    #the labels of subsets A and B are exchanged
    if len(yhat[yhat==1]) > len(yhat[yhat==0]):
        trans = yhat==0
        yhat[yhat==1] = 0
        yhat[trans] = 1
    sum = sum + yhat
    
#When the number of times a sample gets a label of 1 is greater than or equal to 9, 
#it is considered that the label of this sample is 1
final[sum<9] = 0
final[sum>=9] = 1

In [24]:
#Organize the clustering results
clusters = np.unique(final)
subsets={}
for i in clusters:
    subset=[]
    for j in range(len(all_list)):
        if final[j] == i:
            subset.append(all_list[j])
    subsets[i]=subset

In [25]:
#Store positive sample index in each subset
index_lists=[]
#Store the number of positive samples in each subset
posi_cnt =[] 

for i in clusters:
    index_list=[]
    cnt=0
    for j in range(len(subsets[i])):
        if posi_list.__contains__(subsets[i][j]):
            cnt=cnt+1
            index_list.append(j)
    index_lists.append(index_list)
    posi_cnt.append(cnt)

In [29]:
#Find the subset with the least proportion of positive samples
min_per=1
min_idx=0
print('The number of the total sample set is %d, of which the number of positive samples is %d' %(len(all_list),len(posi_list)))
for i in range(len(posi_cnt)):
    t_per=posi_cnt[i]/len(subsets[i])
    print('The total number of samples in subset%d is %d, of which the number of positive samples is %d, and the proportion of positive samples is %f' %(i,len(subsets[i]),posi_cnt[i],t_per))
    if t_per < min_per:
        min_per=t_per
        min_idx=i
print('Subset%d has the least proportion of positive samples, accounting for %f' %(min_idx,min_per))

The number of the total sample set is 189585, of which the number of positive samples is 5430
The total number of samples in subset0 is 121411, of which the number of positive samples is 895, and the proportion of positive samples is 0.007372
The total number of samples in subset1 is 68174, of which the number of positive samples is 4535, and the proportion of positive samples is 0.066521
Subset0 has the least proportion of positive samples, accounting for 0.007372


In [30]:
#Remove the positive samples from the subset with the least number of positive samples, 
#and the remaining unmarked samples in the subset are regarded as negative samples
new_nega=np.delete(subsets[min_idx], index_lists[min_idx], axis=0)
new_nega=new_nega.tolist()

In [31]:
#The negative sample set was divided into two parts according to positive_num
def spliting_negative_data(negative_data, positive_num):
    negative_train_data = negative_data[positive_num:]
    negative_cv_data = negative_data[:positive_num]
    return negative_train_data, negative_cv_data

In [32]:
# Feature selection based on random forest feature importance score
def feature_ranking_by_rf(data, label, sel_fea_num, sel_hp1, sel_hp2):
    fs_rf = RandomForestClassifier(n_estimators=sel_hp1, max_depth=sel_hp2, random_state=0)
    #Training random forest model
    fs_rf.fit(data, label)
    importances = fs_rf.feature_importances_
    #Sort all features in the reverse order of feature importance scores, and return the sorted index value
    indices = np.argsort(importances)[::-1]
    #Extract the top sel_fea_num features with the highest feature importance score
    most_imp = indices[:sel_fea_num]
    return most_imp

In [66]:
#The prediction score of each individual learner to the samples in the test set after applying the feature selection method
def base_probs_fs(X_test, most_imps_list, trained_clfs):
    prob_list = []
    for i,clf in enumerate(trained_clfs):
        #Index value of key features
        most_imp = most_imps_list[i]
        #Extract key features from test set
        X_test_fs = X_test[:,most_imp]
        #Prediction score of individual learner on test set
        prob = clf.predict_proba(X_test_fs)
        prob_list.append(prob[:,1])
    prob_list = np.array(prob_list)
    base_probs = np.transpose(prob_list)
    return base_probs

In [67]:
#The prediction score of each individual learner to the samples in the test set without applying the feature selection method
def base_probs(X_test, trained_clfs):
    prob_list = []
    for clf in trained_clfs:
        prob = clf.predict_proba(X_test)
        prob_list.append(prob[:,1])
    prob_list = np.array(prob_list)
    base_probs = np.transpose(prob_list)
    return base_probs

In [52]:
#XGBoost as individual learners
def base_xgb_learners(base_learner_num):
    clfs = []
    for i in range(base_learner_num):
        clfs.append(XGBClassifier(max_depth=6,learning_rate=0.4,n_estimators=100))
    return clfs

In [53]:
#RandomForest as individual learners
def base_rf_learners(base_learner_num):
    clfs = []
    for i in range(base_learner_num):
        clfs.append(RandomForestClassifier(n_estimators=400, max_depth=40))
    return clfs

In [54]:
#ExtraTrees as individual learners
def base_ert_learners(base_learner_num):
    clfs = []
    for i in range(base_learner_num):
        clfs.append(ExtraTreesClassifier(n_estimators=400, max_depth=20))
    return clfs

In [55]:
#AdaBoost as individual learners
def base_ab_learners(base_learner_num):
    clfs = []
    for i in range(base_learner_num):
        clfs.append(AdaBoostClassifier(n_estimators=50, learning_rate=0.35))
    return clfs

In [56]:
#soft voting strategy
def soft_voting_strategy(base_probs):
    print('\nSoft voting ...\n')
    #Store the final prediction score of each sample
    pred_final = []
    #Store the final prediction result of each sample
    prob_final = []
    for prob in base_probs:
        #Calculate the average of all individual learner prediction scores
        mean_prob = np.mean(prob)
        prob_final.append(mean_prob)
        if mean_prob > 0.5:
            pred_final.append(1)
        else:
            pred_final.append(0)
    return pred_final, prob_final

In [57]:
#Performance calculation
def calculate_performace(num, y_pred, y_prob, y_test):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(num):
        if y_test[index] ==1:
            if y_test[index] == y_pred[index]:
                tp = tp + 1
            else:
                fn = fn + 1
        else:
            if y_test[index] == y_pred[index]:
                tn = tn + 1
            else:
                fp = fp + 1
    acc = float(tp + tn)/num
    try:
        precision = float(tp)/(tp + fp)
        recall = float(tp)/ (tp + fn)
        f1_score = float((2*precision*recall)/(precision+recall))
    except ZeroDivisionError:
        print("You can't divide by 0.")
        precision=recall=f1_score = 100
    AUC = roc_auc_score(y_test, y_prob)
    p, r, _ = precision_recall_curve(y_test,y_prob)
    AUPR = auc(r, p)
    return tp, fp, tn, fn, acc, precision, recall, f1_score, AUC, AUPR

In [71]:
fold_num=5
sel_fea = 'RF'
prop = 0.75
base_learner_num = 10

#Hyper-parameters of Stochastic Forest Model
sel_fea_num = int(prop*len(all_list[0]))
sel_hp1 = 300
sel_hp2 = 30

posi_data = posi_list
nega_data = new_nega
posi_num = len(posi_data)
nega_train_data, nega_cv_data = spliting_negative_data(nega_data, posi_num)

soft_acc_list = []
soft_prec_list = []
soft_recall_list = []
soft_f1_score_list = []
soft_auc_list = []
soft_aupr_list = []

for fold in range(fold_num):
    #Positive training sample
    posi_train_data = np.array([x for i, x in enumerate(posi_data) if i % fold_num != fold])
    #Positive test sample
    posi_test_data = np.array([x for i, x in enumerate(posi_data) if i % fold_num == fold])
    #Positive test sample negative
    nega_test_data = np.array([x for i, x in enumerate(nega_cv_data) if i % fold_num == fold])
    #Generate test set
    X_test = np.concatenate((posi_test_data, nega_test_data))
    y_test = np.concatenate((np.ones(posi_test_data.shape[0]), np.zeros(nega_test_data.shape[0])))
    #Generate base_learner_num individual learners
    base_learners = base_xgb_learners(base_learner_num)
    trained_clfs = []
    most_imps_list = []
    for i in range(base_learner_num):
        print('Round', fold+1, 'cross validation\n')
        print('the', i+1, 'th individual learner training\n')
        sample_num = posi_train_data.shape[0]
        samples = random.sample(nega_train_data, sample_num)
        samples = np.array(samples)
        X_base_train = np.concatenate((posi_train_data, samples))
        y_base_train = np.concatenate((np.ones(posi_train_data.shape[0]), np.zeros(samples.shape[0])))
        if sel_fea == 'RF':
            print('Feature selection in progress\n')
            most_imps = feature_ranking_by_rf(X_base_train, y_base_train, sel_fea_num, sel_hp1, sel_hp2)
            most_imps_list.append(most_imps)
            X_base_train = X_base_train[:,most_imps]
            X_base_test = X_test[:,most_imps]
        else:
            X_base_train = X_base_train
            X_base_test = X_test
        clf = base_learners[i]
        clf.fit(X_base_train,y_base_train)
        trained_clfs.append(clf)
        y_pred = clf.predict(X_base_test)
        y_prob = clf.predict_proba(X_base_test)
        y_prob = y_prob[:,1]
        tp, fp, tn, fn, acc, prec, recall, f1_score, auroc, aupr = calculate_performace(len(y_pred), y_pred, y_prob, y_test) 
        print('the', i+1, 'th individual learner proformance: \n  Acc = \t', acc, 
              '\n  prec = \t', prec, 
              '\n  recall = \t', recall, 
              '\n  f1_score = \t', f1_score, 
              '\n  AUC = \t', auroc, 
              '\n  aupr = \t', aupr, 
              '\n')
        
    if sel_fea == 'RF':
        base_probs = base_probs_fs(X_test, most_imps_list, trained_clfs)
    else:
        base_probs = base_probs(X_test, trained_clfs)
    #soft voting strategy
    pred_final, prob_final = soft_voting_strategy(base_probs)
    soft_tp, soft_fp, soft_tn, soft_fn, soft_acc, soft_prec, soft_recall, soft_f1_score, soft_auc, soft_aupr = calculate_performace(len(pred_final), pred_final, prob_final, y_test)
    print('Round ', fold, 'cross validation proformance after soft voting: \n  Acc = \t', soft_acc, 
          '\n  prec = \t', soft_prec, 
          '\n  recall = \t', soft_recall, 
          '\n  f1_score = \t', soft_f1_score, 
          '\n  AUC = \t', soft_auc, 
          '\n  AUPR = \t', soft_aupr,
          '\n')
    soft_acc_list.append(soft_acc)
    soft_prec_list.append(soft_prec)
    soft_recall_list.append(soft_recall)
    soft_f1_score_list.append(soft_f1_score)
    soft_auc_list.append(soft_auc)
    soft_aupr_list.append(soft_aupr)

soft_acc_arr = np.array(soft_acc_list)
soft_prec_arr = np.array(soft_prec_list)
soft_recall_arr = np.array(soft_recall_list)
soft_f1_score_arr = np.array(soft_f1_score_list)
soft_auc_arr = np.array(soft_auc_list)
soft_aupr_arr = np.array(soft_aupr_list)
        
soft_ave_acc = np.mean(soft_acc_arr)
soft_ave_prec = np.mean(soft_prec_arr)
soft_ave_recall = np.mean(soft_recall_arr)
soft_ave_f1_score = np.mean(soft_f1_score_arr)
soft_ave_auc = np.mean(soft_auc_arr)
soft_ave_aupr = np.mean(soft_aupr_arr)

soft_std_acc = np.std(soft_acc_arr)
soft_std_prec = np.std(soft_prec_arr)
soft_std_recall = np.std(soft_recall_arr)
soft_std_f1_score = np.std(soft_f1_score_arr)
soft_std_auc = np.std(soft_auc_arr)
soft_std_aupr = np.std(soft_aupr_arr)

print('CSMDA Final proformance: \n  Acc = \t', soft_ave_acc, 
      '\n  prec = \t', soft_ave_prec, 
      '\n  recall = \t', soft_ave_recall, 
      '\n  f1_score = \t', soft_ave_f1_score, 
      '\n  AUC = \t', soft_ave_auc, 
      '\n  AUPR = \t', soft_ave_aupr)

Round 1 cross validation

the 1 th individual learner training

Feature selection in progress



C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:34:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
the 1 th individual learner proformance: 
  Acc = 	 0.962707182320442 
  prec = 	 0.96398891966759 
  recall = 	 0.9613259668508287 
  f1_score = 	 0.9626556016597511 
  AUC = 	 0.992465634952128 
  aupr = 	 0.9937499946378503 

Round 1 cross validation

the 2 th individual learner training

Feature selection in progress



C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:34:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
the 2 th individual learner proformance: 
  Acc = 	 0.9636279926335175 
  prec = 	 0.9649122807017544 
  recall = 	 0.9622467771639043 
  f1_score = 	 0.9635776855693868 
  AUC = 	 0.9921586981811029 
  aupr = 	 0.9935656953271996 

Round 1 cross validation

the 3 th individual learner training

Feature selection in progress



...

[20:47:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
the 8 th individual learner proformance: 
  Acc = 	 0.9631675874769797 
  prec = 	 0.9674721189591078 
  recall = 	 0.9585635359116023 
  f1_score = 	 0.9629972247918595 
  AUC = 	 0.9917237297735451 
  aupr = 	 0.9933202873931489 

Round 5 cross validation

the 9 th individual learner training

Feature selection in progress



C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:47:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
the 9 th individual learner proformance: 
  Acc = 	 0.9567219152854513 
  prec = 	 0.9635514018691589 
  recall = 	 0.9493554327808471 
  f1_score = 	 0.9564007421150279 
  AUC = 	 0.9920120129286516 
  aupr = 	 0.9933730505030618 

Round 5 cross validation

the 10 th individual learner training

Feature selection in progress



C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:47:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
the 10 th individual learner proformance: 
  Acc = 	 0.9571823204419889 
  prec = 	 0.9592969472710453 
  recall = 	 0.9548802946593001 
  f1_score = 	 0.9570835256114444 
  AUC = 	 0.9918805897255882 
  aupr = 	 0.9933289250616584 


Soft voting ...

Round  4 cross validation proformance after soft voting: 
  Acc = 	 0.9654696132596685 
  prec = 	 0.9693593314763231 
  recall = 	 0.9613259668508287 
  f1_score = 	 0.9653259361997226 
  AUC = 	 0.9930099813802997 
  AUPR = 	 0.9944126614932424 

CSMDA Final proformance: 
  Acc = 	 0.961049723756906 
  prec = 	 0.9680462618030994 
  recall = 	 0.9535911602209947 
  f1_score = 	 0.9607477063286449 
  AUC = 	 0.9929716566785032 
  AUP

In [73]:
soft_ave_acc=np.round(soft_ave_acc,4)
soft_ave_prec=np.round(soft_ave_prec,4)
soft_ave_recall=np.round(soft_ave_recall,4)
soft_ave_f1_score=np.round(soft_ave_f1_score,4)
soft_ave_auc=np.round(soft_ave_auc,4)
soft_ave_aupr=np.round(soft_ave_aupr,4)
print('  Acc = \t', soft_ave_acc, 
      '\n  prec = \t', soft_ave_prec, 
      '\n  recall = \t', soft_ave_recall, 
      '\n  f1_score = \t', soft_ave_f1_score, 
      '\n  AUC = \t', soft_ave_auc, 
      '\n  aupr = \t', soft_ave_aupr)

  Acc = 	 0.961 
  prec = 	 0.968 
  recall = 	 0.9536 
  f1_score = 	 0.9607 
  AUC = 	 0.993 
  aupr = 	 0.9941


In [75]:
soft_std_acc = np.round(soft_std_acc,4)
soft_std_prec = np.round(soft_std_prec,4)
soft_std_recall = np.round(soft_std_recall,4)
soft_std_f1_score = np.round(soft_std_f1_score,4)
soft_std_auc = np.round(soft_std_auc,4)
soft_std_aupr = np.round(soft_std_aupr,4)
print('  Acc standard deviation= \t', soft_std_acc, 
      '\n  prec standard deviation= \t', soft_std_prec, 
      '\n  recall standard deviation= \t', soft_std_recall, 
      '\n  f1_score standard deviation= \t', soft_std_f1_score, 
      '\n  AUC standard deviation= \t', soft_std_auc, 
      '\n  aupr standard deviation= \t', soft_std_aupr)

  Acc standard deviation= 	 0.0027 
  prec standard deviation= 	 0.0025 
  recall standard deviation= 	 0.0064 
  f1_score standard deviation= 	 0.0029 
  AUC standard deviation= 	 0.0009 
  aupr standard deviation= 	 0.0007
